## 모델 고도화
- Few-Shot Prompting
- Chain-of-Thought Prompting

In [1]:
import math
import re

from tqdm.notebook import tqdm

from eval import get_eval_data, pointwise_eval
from utils import summarize, get_train_data

In [2]:
PROMPT_BASELINE = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

Get Train Data

In [3]:
conv_train = get_train_data()[18]
print(conv_train)

P01: 너 스키장 가본 적 있어?/nP02: 아니 ㅠㅠ 가보고 싶은데 기회가 안돼서 가본 적이 없어/nP02: 너는?/nP01: 나도 없어/nP01: 근데 이번 겨울엔 꼭 가기로 하긴 했지만 ㅜㅜ/nP01: 근데 좀 부끄러울듯 키키/nP02: 너는 뭔가 잘 탈 것 같은데?/nP02: 왜 부끄러워?/nP01: 잘 못타고 계속 넘어질까봐 키키/nP01: 금잔디 *** 같이 탈까봐/nP02: 그건 로보트 아니야?/nP02: ***보단 잘 탈듯/nP01: ㅇㅈ 난 보드는 발에 족쇄찬 느낌일거 같고 스키 배워볼거야/nP01: 넌?/nP02: 근자감인가?/nP02: 나는 잘 탈 것 같은데?/nP01: ^^ 대단한걸~?/nP01: 근데 수영처럼 에너지 소모 커서 컵라면 꿀맛일듯/nP02: 너도 알구나?/nP02: 수영 후 먹는 컵라면이 세상에서 제일 맛있는듯/nP01: 맞어 그리고 바다에서 놀고 나서 먹는 뽕따 진리지/nP02: 뽕따? 그것도 국룰이야?/nP02: 처음 알았네 키키/nP01: 키키 나만의 룰이였나봐/nP01: 근데 나 어릴 때 어린이스포츠 수영 다녀서 살빼려 했는데 더 먹어서 더 쪄썽/nP02: 키키 수영 에너지 소모가 크니까 더 먹게 되는 것 같아 키키/nP02: 나도 그랬음 키키/nP02: 나는 수영 끝나면 맨날 그 옆 매점에서 제일 저렴한 도시락 컵라면 먹은 기억 있어/nP01: 키키 도시락 추억이다/nP01: 난 러시아에서 그거에 마요네즈 뿌려먹는다길래 궁금해서 대학생 때 첨 먹어봄/nP02: 예? 도시락에 마요네즈요?/nP02: 맛 어땠어? 이상할 것 같은데


Few-Shot

In [4]:
summary_sample = """P01과 P02는 스키장 경험이 없지만, P01은 이번 겨울 스키 도전을 계획 중이며 넘어질까 걱정한다.
수영 후 컵라면과 바다에서 뽕따 먹는 추억에 공감하며 어릴 때의 운동과 식습관 이야기를 나눈다.
P01은 대학생 때 러시아 스타일로 도시락에 마요네즈를 뿌려 먹어본 독특한 경험을 공유한다."""

prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 사용자 대화와 요약 예시입니다.
예시 대화:
{conv_train}
예시 요약 결과:
{summary_sample}
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [ ]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash',
    'gpt-3.5-turbo-0125'
]

scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
for model in scores:
    print(scores[model], model)

Chain-of-thought